In [13]:
import pandas as pd

df = pd.read_csv('BTC-USD_17.09.2014-25.10.2023.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [14]:
from datetime import datetime

def prompt_for_date(prompt_message, comparison_date=None, must_be_after=False):
    while True:
        date_input = input(prompt_message)
        try:
            date = datetime.strptime(date_input, "%d-%m-%y")
            if comparison_date is not None:
                if must_be_after and date <= comparison_date:
                    print("The date must be after %s." % comparison_date.strftime("%d-%m-%y"))
                    continue
                elif not must_be_after and date < comparison_date:
                    print("The date must be after or equal to %s." % comparison_date.strftime("%d-%m-%y"))
                    continue
            return date
        except ValueError:
            print("Invalid date format. Please use the format dd-mm-yy.")

min_allowed_start_date = datetime.strptime("06-04-15", "%d-%m-%y")

start_date = prompt_for_date("Enter the start date (dd-mm-yy). Min starting date for this file: 06-04-15: ", min_allowed_start_date, must_be_after=False)
end_date = prompt_for_date("Enter the end date (dd-mm-yy). It must be after the start date: ", start_date, must_be_after=True)

def prompt_for_number(prompt_message, minimum=None, condition=lambda x: True):
    while True:
        try:
            number = float(input(prompt_message))
            if minimum is not None and number <= minimum:
                print("The value must be greater than %s." % minimum)
                continue
            if not condition(number):
                print("The input does not meet the required condition.")
                continue
            return number
        except ValueError:
            print("Please enter a valid number.")

initial_balance = prompt_for_number("Enter the initial balance of the trading bot wallet: ", minimum=0)
profit_percent = prompt_for_number("Enter the % profit (value must be >= 1): ", minimum=1.00) 

start_date_df_ascending_6month = pd.to_datetime('2020-10-01')
end_date_df_ascending_6month  = pd.to_datetime('2021-04-01')
start_date_df_descending_6month = pd.to_datetime('2021-10-01')
end_date_df_descending_6month  = pd.to_datetime('2022-09-01')
start_date_df_stagnation_6month = pd.to_datetime('2015-11-01')
end_date_df_stagnation_6month  = pd.to_datetime('2016-04-01')
start_date_df_combined_1year = pd.to_datetime('2021-11-01')
end_date_df_combined_1year  = pd.to_datetime('2022-05-01')

In [15]:
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
filtered_df_ascending_6month = df[(df['Date'] >= start_date_df_ascending_6month) & (df['Date'] <= end_date_df_ascending_6month)].copy()
filtered_df_descending_6month = df[(df['Date'] >= start_date_df_descending_6month) & (df['Date'] <= end_date_df_descending_6month)]
filtered_df_stagnation_6month = df[(df['Date'] >= start_date_df_stagnation_6month) & (df['Date'] <= end_date_df_stagnation_6month)]
filtered_df_combined_1year = df[(df['Date'] >= start_date_df_combined_1year) & (df['Date'] <= end_date_df_combined_1year)]

filtered_df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume'}, inplace=True)
filtered_df.insert(1, 'time', '00:00')
filtered_df['date'] = filtered_df['date'].dt.strftime('%m/%d/%Y')
filtered_df.drop(columns=['Adj Close'], inplace=True)
filtered_df.to_csv('btcusd_tb1_asc_data.csv', index=False)

filtered_df_ascending_6month.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume'}, inplace=True)
filtered_df_ascending_6month.insert(1, 'time', '00:00')
filtered_df_ascending_6month['date'] = filtered_df_ascending_6month['date'].dt.strftime('%m/%d/%Y')
filtered_df_ascending_6month.drop(columns=['Adj Close'], inplace=True)
filtered_df_ascending_6month.to_csv('btcusd_tb1_asc_data.csv', index=False)

filtered_df_descending_6month.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume'}, inplace=True)
filtered_df_descending_6month.insert(1, 'time', '00:00')
filtered_df_descending_6month['date'] = filtered_df_descending_6month['date'].dt.strftime('%m/%d/%Y')
filtered_df_descending_6month.drop(columns=['Adj Close'], inplace=True)
filtered_df_descending_6month.to_csv('btcusd_tb1_dsc_data.csv', index=False)

filtered_df_stagnation_6month.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume'}, inplace=True)
filtered_df_stagnation_6month.insert(1, 'time', '00:00')
filtered_df_stagnation_6month['date'] = filtered_df_stagnation_6month['date'].dt.strftime('%m/%d/%Y')
filtered_df_stagnation_6month.drop(columns=['Adj Close'], inplace=True)
filtered_df_stagnation_6month.to_csv('btcusd_tb1_stg_data.csv', index=False)

filtered_df_combined_1year.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume'}, inplace=True)
filtered_df_combined_1year.insert(1, 'time', '00:00')
filtered_df_combined_1year['date'] = filtered_df_combined_1year['date'].dt.strftime('%m/%d/%Y')
filtered_df_combined_1year.drop(columns=['Adj Close'], inplace=True)
filtered_df_combined_1year.to_csv('btcusd_tb1_com_data.csv', index=False)

C:\Users\Pako\AppData\Local\Temp\ipykernel_680\3538839484.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume'}, inplace=True)
C:\Users\Pako\AppData\Local\Temp\ipykernel_680\3538839484.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['date'] = filtered_df['date'].dt.strftime('%m/%d/%Y')
C:\Users\Pako\AppData\Local\Temp\ipykernel_680\3538839484.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [16]:
import time
from datetime import datetime

buy_price = None  
balance = initial_balance
trades_asc = []
trades_dsc = []
trades_stg = []
trades_com = []

def trading_bot_1(balance, data, mode, fee):
    trades = []
    holding = 0
    last_buy_balance = 0

    for index, row in data.iterrows():

        date = datetime.strptime(str(row['date']), "%m/%d/%Y")
        unix_format_date = int(time.mktime(date.timetuple()))
        open_price = row['open']
        high_price = row['high']
        low_price = row['low']
        close_price = row['close']

        if holding == 0:
            buy_price = open_price
            real_balance = balance - (fee / balance) * 100
            last_buy_balance = real_balance
            holding = real_balance / buy_price
            buy_date = date
            trades.append([unix_format_date, date, 'BUY', buy_price, open_price, high_price, low_price, close_price, last_buy_balance])
        if holding > 0:
            current_value = holding * close_price
            profit = (current_value - last_buy_balance) / last_buy_balance * 100
            if profit >= profit_percent:
                balance = current_value
                holding = 0
                last_buy_balance =  current_value
                if buy_date == date:
                    sell_date = date.strftime('%Y-%m-%d') + " 23:59:59"
                    unix_format_date = int(time.mktime(datetime.strptime(sell_date, "%Y-%m-%d %H:%M:%S").timetuple()))
                trades.append([unix_format_date, sell_date if buy_date == date else date, 'SELL', close_price, open_price, high_price, low_price, close_price, last_buy_balance])

    final_balance = trades[-1][-1]
    return final_balance, trades

In [17]:
print(f"Initial Balance: ${initial_balance}")

final_balance_tb1, trades = trading_bot_1(balance, filtered_df, 'ASC', 2)
print(f"Final Balance ASC_6M: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%d-%m %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_trades.csv', index=False)

final_balance_tb1, trades_asc = trading_bot_1(balance, filtered_df_ascending_6month, 'ASC', 2)
print(f"Final Balance ASC_6M: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades_asc, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%d-%m %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_asc_trades.csv', index=False)

final_balance_tb1, trades_dsc = trading_bot_1(balance, filtered_df_descending_6month, 'DSC', 2)
print(f"Final Balance ASC_6M: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades_dsc, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%d-%m %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_dsc_trades.csv', index=False)

final_balance_tb1, trades_stg = trading_bot_1(balance, filtered_df_stagnation_6month, 'STG', 2)
print(f"Final Balance ASC_6M: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades_stg, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%d-%m %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_stg_trades.csv', index=False)

final_balance_tb1, trades_com = trading_bot_1(balance, filtered_df_combined_1year, 'COM', 2)
print(f"Final Balance ASC_6M: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades_com, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%d-%m %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_com_trades.csv', index=False)

Initial Balance: $100.0
Final Balance ASC_6M: $269.04659520456084
Final Balance ASC_6M: $349.9791641033652
Final Balance ASC_6M: $132.8883335365215
Final Balance ASC_6M: $131.74200521851506
Final Balance ASC_6M: $103.93081195696537
